In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import Collection
import DBConnector
import datetime

In [ ]:

def loadTrends(nation, disease):
    yesterday = datetime.date.today() - datetime.timedelta(days=1)
    year_ago = yesterday - datetime.timedelta(days=365)
    time = year_ago.strftime('%Y-%m-%d') + ' ' + yesterday.strftime('%Y-%m-%d')

    abbNation = Collection.getAbbNation(nation)
    pytrend = TrendReq()
    if nation =="all":
        pytrend.build_payload(kw_list=[disease], cat=0, timeframe=time)
    else :
        pytrend.build_payload(kw_list=[disease], cat=0, timeframe=time, geo=abbNation)

    contentdata = pytrend.interest_over_time()
    if contentdata.empty:
        return {"overtime": {"times": ['no data'], "values": [0]},
                "region": {"regions": ['no data'], "values": [0]}}

    contentdata = contentdata.iloc[:,:1]
    dataframe = pd.DataFrame(contentdata)
    times = dataframe[disease].index.strftime('%Y%m%d').tolist()
    values = dataframe[disease].values.tolist()
    overTime = {"times": times, "values": values}

    regiondata = pytrend.interest_by_region()
    if regiondata.empty:
        return {"overtime": overTime,
                "region": {"regions": ['no data'], "values": [0]}}

    dataframe2 = pd.DataFrame(regiondata)
    regions = dataframe2[disease].index.tolist()
    values = dataframe2[disease].values.tolist()
    region = {"regions": regions, "values": values}

    result = {"overtime": overTime, "region": region}

    return result

In [ ]:
if __name__ == '__main__':
    nations = Collection.getNationList()
    nations.append('all')
    today = datetime.date.today()

    try:
        for nation in nations:
            for disease in diseaselist:
                yearTrends(nation, disease)
                regionTrends(nation, disease)

    except Exception as e:
        time = datetime.datetime.now()
        exceptionFile = open(EXCEPTION_PATH + today.strftime('%Y%m%d') + '_' + 'exception.txt', 'a', encoding='utf8')
        exceptionFile.write(str(time) + '\t' + e.__str__() + '\n')
